<a href="https://colab.research.google.com/github/Samaraferreira/projeto-intc/blob/main/projeto_intc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Previsão de Desempenho Acadêmico

**Tipo:** Supervisionado (Regressão / Classificação)

**Base de Dados:** Student Performance Dataset (UCI)

**Descrição:** Modelo que estime notas finais de alunos com base em fatores
como estudo, absenteísmo e histórico escolar.

In [ ]:
pip install ucimlrepo

In [ ]:
# ==============================================================================
# 1. CONFIGURAÇÃO E CARREGAMENTO DOS DADOS
# ==============================================================================
# Importação de Bibliotecas Essenciais
import pandas as pd
from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import numpy as np

# Fetch dataset
student_performance = fetch_ucirepo(id=320)

X_raw = student_performance.data.features
y_raw = student_performance.data.targets

print(f"Dimensões Iniciais (Features): {X_raw.shape}")
print("-" * 50)


# ==============================================================================
# 2. ENGENHARIA DE VARIÁVEIS E DEFINIÇÃO DO ALVO (TARGET)
# ==============================================================================

# O target original é G3 (nota final). Vamos criar o target de Classificação.
G3 = y_raw['G3']

# Criando a variável Alvo (Classificação Binária)
# Critério: G3 > Mediana das notas finais.
median_G3 = G3.median()
y = (G3 > median_G3).astype(int)
y.name = 'desempenho_alto' # Renomeia a série para clareza

# Limpeza e Preparação das Features (X)
# O ucimlrepo pode introduzir colunas NaN, que devem ser tratadas.
# A documentação indica que colunas como 'absences' ou 'Medu' podem ter NaNs, mas neste dataset elas são tratadas.
# Iremos preencher NaNs com a moda ou zero, por segurança.
X = X_raw.copy()
# Preenche NaNs em colunas numéricas (embora raras neste dataset)
for col in X.select_dtypes(include=np.number).columns:
    X[col] = X[col].fillna(0)
# Preenche NaNs em colunas categóricas com a moda
for col in X.select_dtypes(include='object').columns:
    X[col] = X[col].fillna(X[col].mode()[0])


print(f"Critério de 'Alto Desempenho': G3 > {median_G3} (Mediana)")
print("Contagem do Desempenho Alto (Target):")
print(y.value_counts())
print("-" * 50)


# ==============================================================================
# 3. DIVISÃO DO CONJUNTO DE DADOS
# ==============================================================================

# 🧱 Divisão em Treino e Teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Conjunto de Treino: {X_train.shape[0]} amostras")
print(f"Conjunto de Teste: {X_test.shape[0]} amostras")
print("-" * 50)


# ==============================================================================
# 4. PRÉ-PROCESSAMENTO E PIPELINE (ColumnTransformer)
# ==============================================================================

# 📝 Definindo Colunas por Tipo
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

# ⚙️ Criando os Transformadores
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# 🛠️ Combinando Transformadores com ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

print("✅ Pré-processamento (ColumnTransformer) definido.")
print("-" * 50)


# ==============================================================================
# 5. TREINAMENTO E AVALIAÇÃO DE MÚLTIPLOS MODELOS
# ==============================================================================

# 🧠 Lista de Modelos a Serem Treinados
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(random_state=42, max_depth=5),
    'SVM (RBF Kernel)': SVC(random_state=42)
}

results = {}

print("Iniciando Treinamento e Avaliação de 3 Modelos...")
for name, classifier in models.items():
    # 🛠️ Criação do Pipeline Completo
    full_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', classifier)
    ])

    # 🚀 Treinamento
    full_pipeline.fit(X_train, y_train)

    # 🧪 Previsão
    y_pred = full_pipeline.predict(X_test)

    # 📊 Avaliação (Métricas Requeridas)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)

    results[name] = {
        'Accuracy': accuracy,
        'Precision (Alto)': report['1']['precision'],
        'Recall (Alto)': report['1']['recall'],
        'F1-Score (Alto)': report['1']['f1-score'],
    }

    print(f"\n--- Resultados para **{name}** ---")
    print(f"Acurácia: {accuracy:.4f}")
    print("Relatório de Classificação:")
    print(classification_report(y_test, y_pred, target_names=['Baixo Desempenho (0)', 'Alto Desempenho (1)'], zero_division=0))

print("-" * 50)


# ==============================================================================
# 6. COMPARAÇÃO FINAL DOS MODELOS
# ==============================================================================

# 📈 Organizando os resultados em um DataFrame para comparação
results_df = pd.DataFrame(results).T

print("\n" + "="*50)
print("     COMPARATIVO FINAL DE DESEMPENHO DOS MODELOS")
print("="*50)
# Ordena por Acurácia
print(results_df.sort_values(by='Accuracy', ascending=False).round(4))


# 🥇 Determinando o Melhor Modelo
best_model_name = results_df['Accuracy'].idxmax()
print(f"\nO modelo com melhor Acurácia é: **{best_model_name}**.")
print("="*50)

✅ Dados Carregados com Sucesso!
Dimensões Iniciais (Features): (649, 30)
--------------------------------------------------
Critério de 'Alto Desempenho': G3 > 12.0 (Mediana)
Contagem do Desempenho Alto (Target):
desempenho_alto
0    373
1    276
Name: count, dtype: int64
--------------------------------------------------
Conjunto de Treino: 519 amostras
Conjunto de Teste: 130 amostras
--------------------------------------------------
✅ Pré-processamento (ColumnTransformer) definido.
--------------------------------------------------
Iniciando Treinamento e Avaliação de 3 Modelos...

--- Resultados para **Logistic Regression** ---
Acurácia: 0.7000
Relatório de Classificação:
                      precision    recall  f1-score   support

Baixo Desempenho (0)       0.76      0.69      0.73        75
 Alto Desempenho (1)       0.63      0.71      0.67        55

            accuracy                           0.70       130
           macro avg       0.70      0.70      0.70       130
   